In [10]:
import numpy as np

In [11]:
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2 + (point1[2] - point2[2]) ** 2)

In [12]:
def initialize_population(pop_size, num_points):
    population = []
    for _ in range(pop_size):
        chromosome = np.random.permutation(num_points)
        population.append(chromosome)
    return population

In [13]:
def fitness(chromosome, points):
    total_distance = 0
    origin = points[0]  
    current_point = origin
    for index in chromosome:
        next_point = points[index + 1] 
        total_distance += calculate_distance(current_point, next_point)
        current_point = next_point
    total_distance += calculate_distance(current_point, origin)
    fitness = 1 / (1 + total_distance) 
    return fitness

In [14]:
def tournament_selection(population, fitness_scores, tournament_size=3):
    tournament = np.random.choice(len(population), tournament_size, replace=False)
    tournament_fitness = [fitness_scores[i] for i in tournament]
    winner_index = tournament[np.argmax(tournament_fitness)]
    return population[winner_index]

In [15]:
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(np.random.choice(range(size), 2, replace=False))  
    offspring = np.empty(size, dtype=parent1.dtype)
    offspring[start:end] = parent1[start:end]  
    fill_values = [item for item in parent2 if item not in offspring[start:end]]
    offspring[:start] = fill_values[:start]
    offspring[end:] = fill_values[start:]
    return offspring


In [16]:
def mutate(chromosome, mutation_rate):
    size = len(chromosome)
    for _ in range(int(size * mutation_rate)):
        idx1, idx2 = np.random.choice(size, 2, replace=False)
        chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]
    return chromosome


In [17]:
def genetic_algorithm(points, pop_size, num_generations, mutation_rate):
    num_points = len(points) - 1  
    population = initialize_population(pop_size, num_points)
    best_solution = None
    best_fitness = float('-inf')

    for generation in range(num_generations):
        fitness_scores = [fitness(chromo, points) for chromo in population]
        generation_best_fitness = max(fitness_scores)
        if generation_best_fitness > best_fitness:
            best_fitness = generation_best_fitness
            best_solution = population[fitness_scores.index(best_fitness)]

        new_population = []
        while len(new_population) < pop_size:
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)
            offspring = crossover(parent1, parent2)
            offspring = mutate(offspring, mutation_rate)
            new_population.append(offspring)

        population = new_population
        print(f"Generation {generation}: Best Fitness = {best_fitness}")

    return best_solution, best_fitness

In [18]:
points_example = [(0,0,0), (10,10,10), (20,20,20), (30,30,30)]  

best_solution, best_fitness = genetic_algorithm(
    points=points_example, pop_size=100, num_generations=100, mutation_rate=0.01
)

Generation 0: Best Fitness = 0.009530794374861805
Generation 1: Best Fitness = 0.009530794374861805
Generation 2: Best Fitness = 0.009530794374861805
Generation 3: Best Fitness = 0.009530794374861805
Generation 4: Best Fitness = 0.009530794374861805
Generation 5: Best Fitness = 0.009530794374861805
Generation 6: Best Fitness = 0.009530794374861805
Generation 7: Best Fitness = 0.009530794374861805
Generation 8: Best Fitness = 0.009530794374861805
Generation 9: Best Fitness = 0.009530794374861805
Generation 10: Best Fitness = 0.009530794374861805
Generation 11: Best Fitness = 0.009530794374861805
Generation 12: Best Fitness = 0.009530794374861805
Generation 13: Best Fitness = 0.009530794374861805
Generation 14: Best Fitness = 0.009530794374861805
Generation 15: Best Fitness = 0.009530794374861805
Generation 16: Best Fitness = 0.009530794374861805
Generation 17: Best Fitness = 0.009530794374861805
Generation 18: Best Fitness = 0.009530794374861805
Generation 19: Best Fitness = 0.009530794